In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' 

import datetime
from datetime import timedelta
from timeit import default_timer as timer

import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tf_netbuilder_ext.extensions import register_tf_netbuilder_extensions
from tensorflow.keras.applications.vgg19 import VGG19
from models import create_openpose_2branches_vgg
from dataset.generators import get_dataset_with_masks
from util import plot_to_image, probe_model_2br_vgg

print("TF:", tf.__version__)
print("Cuda:", tf.test.is_built_with_cuda())
print("GPU:", tf.test.is_gpu_available())

TF: 2.5.0
Cuda: True
GPU: True


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(physical_devices[0], True)

annot_path_train = '../datasets/coco_2017_dataset/annotations/person_keypoints_train2017.json'
img_dir_train = '../datasets/coco_2017_dataset/train2017/'
annot_path_val = '../datasets/coco_2017_dataset/annotations/person_keypoints_val2017.json'
img_dir_val = '../datasets/coco_2017_dataset/val2017/'

checkpoints_folder = './tf_ckpts_2br_vgg/0723_epoch100'
output_weights = 'output_2br_vgg/0723_epoch100/openpose_2br_vgg'


batch_size = 10
lr = 2.5e-5
max_epochs = 100

from_vgg = {
    'VGG/backbone/block1_conv1': 'block1_conv1',
    'VGG/backbone/block1_conv2': 'block1_conv2',
    'VGG/backbone/block2_conv1': 'block2_conv1',
    'VGG/backbone/block2_conv2': 'block2_conv2',
    'VGG/backbone/block3_conv1': 'block3_conv1',
    'VGG/backbone/block3_conv2': 'block3_conv2',
    'VGG/backbone/block3_conv3': 'block3_conv3',
    'VGG/backbone/block3_conv4': 'block3_conv4',
    'VGG/backbone/block4_conv1': 'block4_conv1',
    'VGG/backbone/block4_conv2': 'block4_conv2'
}

In [4]:
def load_initial_vgg19_weights(model):
    """
    """
    print("Loading vgg19 weights...")

    vgg_model = VGG19(include_top=False, weights='imagenet')
    for w in model.weights:

        idx = w.name.find("/kernel")
        if idx > -1:
            nn = w.name[:idx]
            vgg_layer_name = from_vgg.get(nn)
            if vgg_layer_name:
                weights = vgg_model.get_layer(vgg_layer_name).get_weights()[0]
                w.assign(weights)
                print("Loaded VGG19 : " + w.name)
                continue

        idx = w.name.find("/bias")
        if idx > -1:
            nn = w.name[:idx]
            vgg_layer_name = from_vgg.get(nn)
            if vgg_layer_name:
                weights = vgg_model.get_layer(vgg_layer_name).get_weights()[1]
                w.assign(weights)
                print("Loaded VGG19 : " + w.name)


def eucl_loss(y_true, y_pred):
    return tf.reduce_sum(tf.math.squared_difference(y_pred, y_true)) / batch_size / 2


@tf.function
def train_one_step(model, optimizer, x, y_true):
    with tf.GradientTape() as tape:
        y_pred = model(x)

        losses = [eucl_loss(y_true[0], y_pred[0]),
            eucl_loss(y_true[1], y_pred[1]),
            eucl_loss(y_true[0], y_pred[2]),
            eucl_loss(y_true[1], y_pred[3]),
            eucl_loss(y_true[0], y_pred[4]),
            eucl_loss(y_true[1], y_pred[5]),
            eucl_loss(y_true[0], y_pred[6]),
            eucl_loss(y_true[1], y_pred[7]),
            eucl_loss(y_true[0], y_pred[8]),
            eucl_loss(y_true[1], y_pred[9]),
            eucl_loss(y_true[0], y_pred[10]),
            eucl_loss(y_true[1], y_pred[11])]

        total_loss = tf.reduce_sum(losses)

    grads = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return losses, total_loss

In [5]:
def train(ds_train, ds_val, model, optimizer, ckpt, last_epoch, last_step, max_epochs, steps_per_epoch):
    train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
    train_loss_heatmap = tf.keras.metrics.Mean('train_loss_heatmap', dtype=tf.float32)
    train_loss_paf = tf.keras.metrics.Mean('train_loss_paf', dtype=tf.float32)

    val_loss = tf.keras.metrics.Mean('val_loss', dtype=tf.float32)
    val_loss_heatmap = tf.keras.metrics.Mean('val_loss_heatmap', dtype=tf.float32)
    val_loss_paf = tf.keras.metrics.Mean('val_loss_paf', dtype=tf.float32)

    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = 'logs_2br_vgg/gradient_tape/' + current_time + '/train'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    val_log_dir = 'logs_2br_vgg/gradient_tape/' + current_time + '/val'
    val_summary_writer = tf.summary.create_file_writer(val_log_dir)

    output_paf_idx = 10
    output_heatmap_idx = 11

    # determine start epoch in case the training has been stopped manually and resumed

    resume = last_step != 0 and (steps_per_epoch - last_step) != 0
    if resume:
        start_epoch = last_epoch
    else:
        start_epoch = last_epoch + 1

    # start processing

    for epoch in range(start_epoch, max_epochs + 1, 1):

        start = timer()

        print("Start processing epoch {}".format(epoch))

        # set the initial step index depending on if you resumed the processing

        if resume:
            step = last_step + 1
            data_iter = ds_train.skip(last_step)
            print(f"Skipping {last_step} steps (May take a few minutes)...")
            resume = False
        else:
            step = 0
            data_iter = ds_train

        # process steps

        for x, y in data_iter:

            step += 1

            losses, total_loss = train_one_step(model, optimizer, x, y)

            train_loss(total_loss)
            train_loss_heatmap(losses[output_heatmap_idx])
            train_loss_paf(losses[output_paf_idx])

            if step % 10 == 0:
                tf.print('Epoch', epoch, f'Step {step}/{steps_per_epoch}',
                         '(1)', losses[0], '(2)', losses[1], '(3)', losses[2], '(4)', losses[3],
                         '(5)', losses[4], '(6)', losses[5], '(7)', losses[6], '(8)', losses[7],
                         '(9)', losses[8], '(10)', losses[9], '(11)', losses[10], '(12)', losses[11],
                         'Total loss', total_loss)

                with train_summary_writer.as_default():
                    summary_step = (epoch - 1) * steps_per_epoch + step - 1
                    tf.summary.scalar('loss', train_loss.result(), step=summary_step)
                    tf.summary.scalar('loss_heatmap', train_loss_heatmap.result(), step=summary_step)
                    tf.summary.scalar('loss_paf', train_loss_paf.result(), step=summary_step)

            if step % 100 == 0:
                figure = probe_model_2br_vgg(model, test_img_path="resources/ski_368.jpg")
                with train_summary_writer.as_default():
                    tf.summary.image("Test prediction", plot_to_image(figure), step=step)

            if step % 1000 == 0:
                ckpt.step.assign(step)
                ckpt.epoch.assign(epoch)
                save_path = manager.save()
                print("Saved checkpoint for step {}: {}".format(step, save_path))

            if step >= steps_per_epoch:
                break

        print("Completed epoch {}. Saving weights...".format(epoch))
        model.save_weights(output_weights, overwrite=True)

        # save checkpoint at the end of an epoch

        ckpt.step.assign(step)
        ckpt.epoch.assign(epoch)
        manager.save()

        # reset metrics every epoch

        train_loss.reset_states()
        train_loss_heatmap.reset_states()
        train_loss_paf.reset_states()

        end = timer()

        print("Epoch training time: " + str(timedelta(seconds=end - start)))

        # calculate validation loss

        print("Calculating validation losses...")

        for val_step, (x_val, y_val_true) in enumerate(ds_val):

            if val_step % 1000 == 0:
                print(f"Validation step {val_step} ...")

            y_val_pred = model(x_val)

            losses = [eucl_loss(y_val_true[0], y_val_pred[0]),
                      eucl_loss(y_val_true[1], y_val_pred[1]),
                      eucl_loss(y_val_true[0], y_val_pred[2]),
                      eucl_loss(y_val_true[1], y_val_pred[3]),
                      eucl_loss(y_val_true[0], y_val_pred[4]),
                      eucl_loss(y_val_true[1], y_val_pred[5]),
                      eucl_loss(y_val_true[0], y_val_pred[6]),
                      eucl_loss(y_val_true[1], y_val_pred[7]),
                      eucl_loss(y_val_true[0], y_val_pred[8]),
                      eucl_loss(y_val_true[1], y_val_pred[9]),
                      eucl_loss(y_val_true[0], y_val_pred[10]),
                      eucl_loss(y_val_true[1], y_val_pred[11])]

            total_loss = tf.reduce_sum(losses)
            val_loss(total_loss)
            val_loss_heatmap(losses[output_heatmap_idx])
            val_loss_paf(losses[output_paf_idx])

        val_loss_res = val_loss.result()
        val_loss_heatmap_res = val_loss_heatmap.result()
        val_loss_paf_res = val_loss_paf.result()

        print(f'Validation losses for epoch: {epoch} : Loss paf {val_loss_paf_res}, Loss heatmap '
              f'{val_loss_heatmap_res}, Total loss {val_loss_res}')

        with val_summary_writer.as_default():
            tf.summary.scalar('val_loss', val_loss_res, step=epoch)
            tf.summary.scalar('val_loss_heatmap', val_loss_heatmap_res, step=epoch)
            tf.summary.scalar('val_loss_paf', val_loss_paf_res, step=epoch)
        val_loss.reset_states()
        val_loss_heatmap.reset_states()
        val_loss_paf.reset_states()

In [ ]:
if __name__ == '__main__':

    # registering custom blocks types

    register_tf_netbuilder_extensions()

    # loading datasets

    ds_train, ds_train_size = get_dataset_with_masks(annot_path_train, img_dir_train, batch_size)
    ds_val, ds_val_size = get_dataset_with_masks(annot_path_val, img_dir_val, batch_size, strict=True)

    print(f"Training samples: {ds_train_size} , Validation samples: {ds_val_size}")

    steps_per_epoch = ds_train_size // batch_size
    steps_per_epoch_val = ds_val_size // batch_size

    # creating model, optimizers etc

    # model = create_openpose_2branches_vgg(pretrained=False, training=True)
    model = create_openpose_2branches_vgg(pretrained=True, training=False)

    optimizer = RMSprop(lr)

    # loading previous state if required

    ckpt = tf.train.Checkpoint(step=tf.Variable(0), epoch=tf.Variable(0), optimizer=optimizer, net=model)
    manager = tf.train.CheckpointManager(ckpt, checkpoints_folder, max_to_keep=3)
    ckpt.restore(manager.latest_checkpoint)
    last_step = int(ckpt.step)
    last_epoch = int(ckpt.epoch)

    if manager.latest_checkpoint:
        print(f"Restored from {manager.latest_checkpoint}")
        print(f"Resumed from epoch {last_epoch}, step {last_step}")
    else:
        print("Initializing from scratch.")

        load_initial_vgg19_weights(model)

    # training loop

    train(ds_train, ds_val, model, optimizer, ckpt, last_epoch, last_step, max_epochs, steps_per_epoch)
    print("Done!")

loading annotations into memory...
Done (t=4.88s)
creating index...
index created!
Loading image annot 0/118287
Loading image annot 1000/118287
Loading image annot 8000/118287
Loading image annot 10000/118287
Loading image annot 11000/118287
Loading image annot 12000/118287
Loading image annot 14000/118287
Loading image annot 16000/118287
Loading image annot 19000/118287
Loading image annot 22000/118287
Loading image annot 24000/118287
Loading image annot 31000/118287
Loading image annot 37000/118287
Loading image annot 38000/118287
Loading image annot 40000/118287
Loading image annot 47000/118287
Loading image annot 60000/118287
Loading image annot 63000/118287
Loading image annot 65000/118287
Loading image annot 66000/118287
Loading image annot 69000/118287
Loading image annot 72000/118287
Loading image annot 73000/118287
Loading image annot 75000/118287
Loading image annot 76000/118287
Loading image annot 80000/118287
Loading image annot 82000/118287
Loading image annot 84000/118287

Epoch 25 Step 4280/5304 (1) 61.3011246 (2) 21.2362251 (3) 51.0205956 (4) 17.7279186 (5) 48.4964447 (6) 16.1628189 (7) 48.6979179 (8) 15.2442417 (9) 47.4683914 (10) 15.433054 (11) 47.421 (12) 15.1993055 Total loss 405.409027
Epoch 25 Step 4290/5304 (1) 56.1847954 (2) 20.9876633 (3) 43.3984451 (4) 17.4821091 (5) 41.5695763 (6) 16.7016506 (7) 40.7282677 (8) 16.8577938 (9) 40.6689224 (10) 15.8043518 (11) 40.2337036 (12) 15.891078 Total loss 366.508362
Epoch 25 Step 4300/5304 (1) 79.1421127 (2) 32.9934921 (3) 70.5625763 (4) 27.2971096 (5) 66.1787567 (6) 26.7795868 (7) 67.0709381 (8) 26.7615814 (9) 66.9925232 (10) 26.3355312 (11) 66.7444077 (12) 26.2191906 Total loss 583.07782
Epoch 25 Step 4310/5304 (1) 67.7361221 (2) 23.6660671 (3) 54.1265259 (4) 18.7482376 (5) 51.4764671 (6) 17.8240929 (7) 50.646225 (8) 16.8946304 (9) 50.745491 (10) 16.7754211 (11) 50.0996666 (12) 16.2293034 Total loss 434.968231
Epoch 25 Step 4320/5304 (1) 59.162117 (2) 20.3565731 (3) 47.1593437 (4) 18.2507401 (5) 45.693

Epoch 25 Step 4650/5304 (1) 60.9943619 (2) 22.0638466 (3) 49.736866 (4) 18.3500404 (5) 47.3817177 (6) 16.7655983 (7) 46.8702431 (8) 16.2814903 (9) 46.6683922 (10) 16.0836811 (11) 47.0191956 (12) 16.2287769 Total loss 404.444214
Epoch 25 Step 4660/5304 (1) 59.6241531 (2) 20.4942226 (3) 46.1633 (4) 17.2333679 (5) 41.6587677 (6) 15.9330339 (7) 42.1323853 (8) 15.2472143 (9) 41.1278458 (10) 15.2753162 (11) 40.6392479 (12) 15.2588854 Total loss 370.78775
Epoch 25 Step 4670/5304 (1) 48.6060181 (2) 17.1720333 (3) 39.1895065 (4) 13.6269169 (5) 36.9451714 (6) 12.1909637 (7) 34.9394531 (8) 11.3153048 (9) 34.7249 (10) 11.0438604 (11) 34.3379784 (12) 10.6326742 Total loss 304.724792
Epoch 25 Step 4680/5304 (1) 86.2835083 (2) 23.5027733 (3) 76.8612 (4) 22.2786064 (5) 72.6100159 (6) 20.8641033 (7) 71.1186676 (8) 20.0185413 (9) 69.0533371 (10) 20.2404652 (11) 68.7351913 (12) 19.94561 Total loss 571.512
Epoch 25 Step 4690/5304 (1) 62.809967 (2) 20.1142311 (3) 52.8152771 (4) 16.4430637 (5) 50.9229851 (6

Epoch 25 Step 5010/5304 (1) 57.3518867 (2) 17.7124119 (3) 49.1056442 (4) 15.4703703 (5) 46.4880447 (6) 14.6230211 (7) 46.8607597 (8) 13.3392334 (9) 45.3278275 (10) 13.4225645 (11) 44.5601807 (12) 13.2244768 Total loss 377.48645
Epoch 25 Step 5020/5304 (1) 76.0616 (2) 28.9671764 (3) 64.695343 (4) 23.0842075 (5) 58.7407799 (6) 21.2228603 (7) 59.834938 (8) 20.1969376 (9) 58.1154861 (10) 21.4485989 (11) 58.3768883 (12) 21.6047325 Total loss 512.349548
Epoch 25 Step 5030/5304 (1) 61.263134 (2) 21.6978512 (3) 48.5018501 (4) 15.7699537 (5) 46.4265442 (6) 15.138629 (7) 45.2390251 (8) 14.3140478 (9) 43.6827316 (10) 14.5232773 (11) 44.3562965 (12) 13.9830198 Total loss 384.896362
Epoch 25 Step 5040/5304 (1) 83.3763275 (2) 27.1738834 (3) 74.796669 (4) 26.4338837 (5) 73.3028564 (6) 25.0205288 (7) 71.607048 (8) 24.818 (9) 70.6829147 (10) 25.1998196 (11) 71.3970795 (12) 24.8491631 Total loss 598.658203
Epoch 25 Step 5050/5304 (1) 42.2989616 (2) 11.1903906 (3) 39.721302 (4) 10.0800867 (5) 39.3072 (6)

Epoch 26 Step 70/5304 (1) 72.378685 (2) 20.8613491 (3) 60.2799873 (4) 17.7053947 (5) 56.528614 (6) 16.0049343 (7) 57.0198784 (8) 15.9334364 (9) 56.8043289 (10) 15.6175327 (11) 57.3163948 (12) 15.6233778 Total loss 462.073914
Epoch 26 Step 80/5304 (1) 99.3044052 (2) 36.5957031 (3) 90.3499 (4) 31.5453777 (5) 87.6300583 (6) 31.4960823 (7) 89.3879242 (8) 31.1132717 (9) 89.2557449 (10) 30.6505985 (11) 90.5229416 (12) 31.1645756 Total loss 739.016602
Epoch 26 Step 90/5304 (1) 89.426506 (2) 32.2987709 (3) 74.9564438 (4) 25.5437527 (5) 70.7786636 (6) 24.1174278 (7) 69.6128159 (8) 23.214241 (9) 68.9059296 (10) 23.277092 (11) 68.6970596 (12) 23.5429745 Total loss 594.371704
Epoch 26 Step 100/5304 (1) 68.1409607 (2) 21.4967899 (3) 52.6454544 (4) 18.2411232 (5) 52.4438248 (6) 16.0244389 (7) 50.7327652 (8) 15.9362059 (9) 50.3072929 (10) 15.9575825 (11) 50.9245872 (12) 15.5312624 Total loss 428.382294
Epoch 26 Step 110/5304 (1) 65.181633 (2) 21.933424 (3) 50.0524864 (4) 17.6540012 (5) 44.9115105 (6)

...

Epoch 27 Step 2840/5304 (1) 76.8471298 (2) 23.7331066 (3) 64.5376205 (4) 20.1038723 (5) 61.1680603 (6) 18.1674519 (7) 60.0459671 (8) 18.139143 (9) 59.9102135 (10) 18.0804043 (11) 60.0461731 (12) 17.7367382 Total loss 498.5159
Epoch 27 Step 2850/5304 (1) 84.620636 (2) 29.7766781 (3) 70.2799301 (4) 25.5535336 (5) 67.698288 (6) 24.9139309 (7) 68.1386871 (8) 24.5268707 (9) 67.5140228 (10) 23.526701 (11) 67.3659286 (12) 23.8342762 Total loss 577.749512
Epoch 27 Step 2860/5304 (1) 46.1463242 (2) 12.8781605 (3) 38.3637581 (4) 10.4410849 (5) 36.9970589 (6) 9.27347565 (7) 36.6489449 (8) 8.98082638 (9) 35.7365189 (10) 8.67194271 (11) 35.5238228 (12) 8.35043144 Total loss 288.01236
Epoch 27 Step 2870/5304 (1) 54.6616325 (2) 19.7120571 (3) 48.6680565 (4) 17.0567665 (5) 45.3628082 (6) 16.5885162 (7) 45.2075386 (8) 16.6375751 (9) 45.6083946 (10) 16.0279446 (11) 44.0774422 (12) 16.1587791 Total loss 385.767517
Epoch 27 Step 2880/5304 (1) 57.1835747 (2) 21.5927086 (3) 46.5871582 (4) 18.2345276 (5) 45.

Epoch 27 Step 3200/5304 (1) 57.8802071 (2) 17.9375038 (3) 45.0729942 (4) 14.1392641 (5) 42.224411 (6) 12.1864805 (7) 41.9093132 (8) 11.3566122 (9) 39.9566574 (10) 11.1703405 (11) 40.3228722 (12) 10.7662487 Total loss 344.922913
Epoch 27 Step 3210/5304 (1) 56.1795158 (2) 21.50737 (3) 47.0628433 (4) 18.5088806 (5) 44.858757 (6) 17.2125549 (7) 43.8920441 (8) 16.9442806 (9) 43.9732971 (10) 16.7368298 (11) 43.666256 (12) 16.6889229 Total loss 387.231567
Epoch 27 Step 3220/5304 (1) 80.0625534 (2) 24.8832226 (3) 63.2953606 (4) 20.4747 (5) 57.2633743 (6) 19.4522133 (7) 56.3106766 (8) 18.2168503 (9) 55.7919388 (10) 17.8282452 (11) 56.179882 (12) 17.6759129 Total loss 487.434967
Epoch 27 Step 3230/5304 (1) 64.1988373 (2) 22.7157536 (3) 59.0812073 (4) 21.0939484 (5) 57.1350365 (6) 20.7149296 (7) 56.1867561 (8) 20.2001266 (9) 54.9762573 (10) 19.7474575 (11) 54.4941483 (12) 19.348959 Total loss 469.893433
Epoch 27 Step 3240/5304 (1) 76.316658 (2) 24.373579 (3) 62.7492485 (4) 19.8457813 (5) 57.15073

Epoch 27 Step 3570/5304 (1) 62.6544151 (2) 21.7476826 (3) 50.4872284 (4) 18.4694405 (5) 47.4959869 (6) 16.8866978 (7) 45.9293175 (8) 16.1677303 (9) 45.4627953 (10) 15.8880482 (11) 45.0987625 (12) 15.4756517 Total loss 401.763763
Epoch 27 Step 3580/5304 (1) 65.7325287 (2) 22.7403851 (3) 55.3941765 (4) 20.1991463 (5) 53.1127701 (6) 19.1137829 (7) 51.8857 (8) 17.9510231 (9) 51.1206474 (10) 17.6827946 (11) 51.1510811 (12) 17.6192646 Total loss 443.703308
Epoch 27 Step 3590/5304 (1) 50.3836212 (2) 14.6494312 (3) 41.2499962 (4) 11.6656666 (5) 38.3485413 (6) 10.7603359 (7) 37.7962875 (8) 10.4156399 (9) 37.2557182 (10) 10.3040047 (11) 36.7476768 (12) 10.2013235 Total loss 309.778259
Epoch 27 Step 3600/5304 (1) 67.4154358 (2) 21.1574802 (3) 54.7183952 (4) 17.8841248 (5) 52.5778809 (6) 17.0919971 (7) 51.3006592 (8) 16.7518749 (9) 52.1653252 (10) 16.7571182 (11) 51.5528336 (12) 16.900053 Total loss 436.273193
Epoch 27 Step 3610/5304 (1) 105.397583 (2) 40.2098198 (3) 92.3016815 (4) 36.1400566 (5) 

Epoch 27 Step 3940/5304 (1) 65.9468079 (2) 19.8551979 (3) 54.3908081 (4) 17.9436474 (5) 52.9394112 (6) 16.4546375 (7) 50.8042793 (8) 16.7565975 (9) 51.7242317 (10) 16.0842896 (11) 51.0271263 (12) 15.9147024 Total loss 429.841705
Epoch 27 Step 3950/5304 (1) 71.9348221 (2) 24.3001347 (3) 63.7855949 (4) 21.4020634 (5) 60.1149101 (6) 20.2362404 (7) 57.7934456 (8) 19.2499752 (9) 56.9324532 (10) 19.2597351 (11) 57.050415 (12) 17.989336 Total loss 490.049103
Epoch 27 Step 3960/5304 (1) 68.4787 (2) 20.9966564 (3) 60.330555 (4) 19.0969048 (5) 57.9664078 (6) 19.7312984 (7) 57.2199287 (8) 18.6856136 (9) 57.1832657 (10) 18.2664 (11) 57.4569168 (12) 17.758482 Total loss 473.171112
Epoch 27 Step 3970/5304 (1) 65.856041 (2) 21.4309597 (3) 58.0021248 (4) 18.3061447 (5) 56.2012711 (6) 17.6121426 (7) 56.796627 (8) 17.127552 (9) 56.7432327 (10) 17.2237282 (11) 55.9479637 (12) 17.0367184 Total loss 458.284546
Epoch 27 Step 3980/5304 (1) 89.2171631 (2) 28.6305847 (3) 79.6933594 (4) 26.9699841 (5) 79.765853

Epoch 27 Step 4300/5304 (1) 62.3099174 (2) 23.951128 (3) 52.9602661 (4) 20.0755749 (5) 50.4614563 (6) 19.5388317 (7) 50.4306526 (8) 18.3962269 (9) 49.8873482 (10) 18.6028442 (11) 50.2882881 (12) 18.6001186 Total loss 435.502686
Epoch 27 Step 4310/5304 (1) 68.7518082 (2) 23.9623394 (3) 58.3853226 (4) 20.6081543 (5) 56.0986748 (6) 19.6049366 (7) 55.1608696 (8) 19.2806 (9) 55.4109192 (10) 19.1110268 (11) 54.6679153 (12) 18.9609184 Total loss 470.003479
Epoch 27 Step 4320/5304 (1) 60.1409492 (2) 20.2290478 (3) 52.0182915 (4) 17.3458233 (5) 48.9447136 (6) 16.8533382 (7) 49.0696564 (8) 16.46138 (9) 48.5478554 (10) 16.3613415 (11) 48.2011871 (12) 16.1609974 Total loss 410.334534
Epoch 27 Step 4330/5304 (1) 62.1085892 (2) 22.2209911 (3) 55.6501884 (4) 19.5357857 (5) 52.7095718 (6) 19.0219727 (7) 52.7206535 (8) 18.6787281 (9) 51.775631 (10) 18.7538948 (11) 51.1626701 (12) 18.3851109 Total loss 442.723785
Epoch 27 Step 4340/5304 (1) 61.9549141 (2) 22.6537228 (3) 54.7476768 (4) 22.2691021 (5) 54.